<a href="https://colab.research.google.com/github/neel26desai/cmpe297_assignment2/blob/main/llama_factory/DPO_Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
%cd /content/drive/MyDrive
%rm -rf LLaMA-Factory
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory
%ls
!pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1
!pip uninstall -y jax
!pip install -e .[torch,bitsandbytes,liger-kernel]

/content/drive/MyDrive
Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 315, done.
remote: Counting objects: 100% (315/315), done.
remote: Compressing objects: 100% (245/245), done.
remote: Total 315 (delta 80), reused 159 (delta 57), pack-reused 0 (from 0)
Receiving objects: 100% (315/315), 8.94 MiB | 16.11 MiB/s, done.
Resolving deltas: 100% (80/80), done.
Updating files: 100% (257/257), done.
/content/drive/MyDrive/LLaMA-Factory
assets/       docker/      LICENSE      pyproject.toml  requirements.txt  src/
CITATION.cff  evaluation/  Makefile     README.md       scripts/          tests/
data/         examples/    MANIFEST.in  README_zh.md    setup.py
Obtaining file:///content/drive/MyDrive/LLaMA-Factory
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for llamafactory (pyproject.toml) ... done
  

In [3]:
!pip install git+https://github.com/huggingface/transformers
!pip install liger-kernel

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-fzo4b662
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-fzo4b662
  Resolved https://github.com/huggingface/transformers to commit 8bd2b1e8c23234cd607ca8d63f53c1edfea27462
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.45.0.dev0-py3-none-any.whl size=9732511 sha256=3632bf0293bc3900549794feedbe9a8ee4b88e6dbb073cba1d0d080cdbca266b
  Stored in directory: /tmp/pip-ephem-wheel-cache-3emg9a8b/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import torch
try:
  assert torch.cuda.is_available() is True
except AssertionError:
  print("Please set up a GPU before using LLaMA Factory: https://medium.com/mlearning-ai/training-yolov4-on-google-colab-316f8fff99c6")

In [5]:
import json

%cd /content/drive/MyDrive/LLaMA-Factory

NAME = "Llama-3"
AUTHOR = "LLaMA Factory"

with open("data/identity.json", "r", encoding="utf-8") as f:
  dataset = json.load(f)

for sample in dataset:
  sample["output"] = sample["output"].replace("{{"+ "name" + "}}", NAME).replace("{{"+ "author" + "}}", AUTHOR)

with open("data/identity.json", "w", encoding="utf-8") as f:
  json.dump(dataset, f, indent=2, ensure_ascii=False)

/content/drive/MyDrive/LLaMA-Factory


In [1]:
from google.colab import userdata
HF_TOKEN2 = userdata.get('HF_TOKEN2')

In [2]:
!huggingface-cli login --token $HF_TOKEN2 --add-to-git-credential

Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [8]:
import json

args = {
    ### model
    "model_name_or_path": "Qwen/Qwen2-VL-7B-Instruct",

    ### method
    "stage": "dpo",
    "do_train": True,
    "finetuning_type": "lora",
    "lora_target": "all",
    "pref_beta": 0.1,
    "pref_loss": "sigmoid",  # choices: [sigmoid (dpo), orpo, simpo]

    ### dataset
    "dataset": "rlhf_v",
    "template": "qwen2_vl",
    "cutoff_len": 1024,
    "max_samples": 1000,
    "overwrite_cache": True,
    "preprocessing_num_workers": 16,

    ### output
    "output_dir": "/content/drive/MyDrive/saves/qwen2_vl-7b/lora/dpo",
    "logging_steps": 10,
    "save_steps": 500,
    "plot_loss": True,
    "overwrite_output_dir": True,

    ### train
    "per_device_train_batch_size": 1,
    "gradient_accumulation_steps": 8,
    "learning_rate": 5.0e-6,
    "num_train_epochs": 3.0,
    "lr_scheduler_type": "cosine",
    "warmup_ratio": 0.1,
    "bf16": True,
    "ddp_timeout": 180000000,

    ### eval
    "val_size": 0.1,
    "per_device_eval_batch_size": 1,
    "eval_strategy": "steps",
    "eval_steps": 500
}

json.dump(args, open("train_llama3.json", "w", encoding="utf-8"), indent=2)


In [9]:
%cd /content/drive/MyDrive/LLaMA-Factory/

!llamafactory-cli train train_llama3.json

/content/drive/MyDrive/LLaMA-Factory
2024-09-15 19:24:39.870898: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-15 19:24:39.889186: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-15 19:24:39.910156: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-15 19:24:39.916704: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-15 19:24:39.932057: I tens

In [2]:
!mkdir -p /content/drive/MyDrive/saves/qwen2_vl-7b/lora/dpo

In [3]:
!cp -r /conten/drive/MyDrive/saves/qwen2_vl-7b/lora/dpo /content/drive/MyDrive/saves/qwen2_vl-7b/lora/dpo

## Inference using the fine tuned model

In [12]:
!pip install llamafactory

In [ ]:
from llamafactory.chat import ChatModel
from llamafactory.extras.misc import torch_gc

%cd /content/LLaMA-Factory/

args = dict(
  model_name_or_path="Qwen/Qwen2-VL-7B-Instruct", # use bnb-4bit-quantized Llama-3-8B-Instruct model
  adapter_name_or_path="/conten/drive/MyDrive/saves/qwen2_vl-7b/lora/dpo",            # load the saved LoRA adapters
  template="qwen2_vl",                     # same to the one in training
  finetuning_type="lora",                  # same to the one in training
  quantization_bit=4,                    # load 4-bit quantized model
)
chat_model = ChatModel(args)

messages = []
print("Welcome to the CLI application, use `clear` to remove the history, use `exit` to exit the application.")
while True:
  query = input("\nUser: ")
  if query.strip() == "exit":
    break
  if query.strip() == "clear":
    messages = []
    torch_gc()
    print("History has been removed.")
    continue

  messages.append({"role": "user", "content": query})
  print("Assistant: ", end="", flush=True)

  response = ""
  for new_text in chat_model.stream_chat(messages):
    print(new_text, end="", flush=True)
    response += new_text
  print()
  messages.append({"role": "assistant", "content": response})

torch_gc()